<a href="https://colab.research.google.com/github/SootyOwl/pyhnefatafl/blob/main/neural_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import logging

import coloredlogs
from alphazero.coach import Coach
from hnefatafl.game import HnefataflGame as Game
from hnefatafl.ai import NNetWrapper as nn
from alphazero.utils import dotdict

log = logging.getLogger(__name__)
coloredlogs.install(level="INFO", logger=log)

In [11]:
args = dotdict(
    {
    'numIters': 1,
    'numEps': 1,                # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,        # 
    'updateThreshold': 0.6,     # During arena playoff, new neural net will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    'numMCTSSims': 10,          # Number of games moves for MCTS to simulate.
    'arenaCompare': 40,         # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': 1,                 # The amount of exploration to allow in the MCTS tree.

    'checkpoint': './temp/',
    'load_model': False,
    'load_folder_file': ('/dev/models/8x100x50','best.pth.tar'),
    'numItersForTrainExamplesHistory': 20,
    }
)


In [12]:
def main():
    log.info('Loading %s...', Game.__name__)
    g = Game()

    log.info('Loading %s...', nn.__name__)
    nnet = nn(g)

    if args.load_model:
        log.info('Loading checkpoint "%s/%s"...', args.load_folder_file[0], args.load_folder_file[1])
        nnet.load_checkpoint(args.load_folder_file[0], args.load_folder_file[1])
    else:
        log.warning('Not loading a checkpoint!')

    log.info('Loading the Coach...')
    c = Coach(g, nnet, args)

    if args.load_model:
        log.info("Loading 'trainExamples' from file...")
        c.loadTrainExamples()

    log.info('Starting the learning process 🎉')
    c.learn()

In [13]:
main()

2023-02-09 12:14:27 saiph __main__[516631] INFO Loading HnefataflGame...
2023-02-09 12:14:27 saiph __main__[516631] INFO Loading NNetWrapper...


The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_16), but
are not present in its tracked objects:
  <tf.Variable 'batch_normalization_24/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'batch_normalization_24/beta:0' shape=(512,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_17), but
are not present in its tracked objects:
  <tf.Variable 'batch_normalization_25/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'batch_normalization_25/beta:0' shape=(512,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables we

2023-02-09 12:14:29 saiph __main__[516631] WARNING Not loading a checkpoint!
2023-02-09 12:14:29 saiph __main__[516631] INFO Loading the Coach...


The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_20), but
are not present in its tracked objects:
  <tf.Variable 'batch_normalization_30/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'batch_normalization_30/beta:0' shape=(512,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_21), but
are not present in its tracked objects:
  <tf.Variable 'batch_normalization_31/gamma:0' shape=(512,) dtype=float32>
  <tf.Variable 'batch_normalization_31/beta:0' shape=(512,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables we

2023-02-09 12:14:30 saiph __main__[516631] INFO Starting the learning process 🎉
Self Play:   0%|          | 0/1 [00:46<?, ?it/s]


RecursionError: maximum recursion depth exceeded